## LSTM Attempt

In [1]:
import numpy as np
import pandas as pd
# import tensorflow as tf
# from tf.keras.models import Sequential
# from tf.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [3]:
# files = ['balanced_data/BK7610_balanced_250hz.parquet', 
#          'balanced_data/BU4707_balanced_250hz.parquet', 
#          'balanced_data/CC6740_balanced_250hz.parquet',
#          'balanced_data/DC6359_balanced_250hz.parquet',
#          'balanced_data/DK3500_balanced_250hz.parquet',
#          'balanced_data/HV0618_balanced_250hz.parquet',
#          'balanced_data/JB3156_balanced_250hz.parquet',
#          'balanced_data/JR8022_balanced_250hz.parquet',
#          'balanced_data/MC7070_balanced_250hz.parquet',
#          'balanced_data/MJ8002_balanced_250hz.parquet',
#          'balanced_data/PC6771_balanced_250hz.parquet',
#          ]

In [4]:
# test_files = ['balanced_data/SA0297_balanced_250hz.parquet',
#          'balanced_data/SF3079_balanced_250hz.parquet']

In [5]:
merged = pd.read_parquet('merged_data.parquet')
merged = merged.drop(columns=['time', 'pid'])

In [ ]:
# dfs = []
# for file in files:
#     try:
#         df = pd.read_parquet(file)
#         dfs.append(df)
#         print(f"Successfully read file: {file}")
#     except Exception as e:
#         print(f"Error reading file {file}: {e}")


# combined_df = pd.concat(dfs, ignore_index=True)

In [6]:
# Generating sample accelerometer and TAC data
# Replace this with your actual data
# Assuming you have a DataFrame with columns 'x', 'y', 'z', and 'tac'
data = merged

# Normalizing data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)


In [ ]:

# Convert data to sequences for LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length, :-1])  # Exclude last column (TAC) for input
        y.append(data[i + seq_length, -1])     # Last column (TAC) for output
    return np.array(X), np.array(y)

seq_length = 50
X, y = create_sequences(scaled_data, seq_length)


In [15]:
# x,y = scaled_data[:, :3], scaled_data[:, 3]

In [16]:

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
import tensorflow as tf

2024-04-17 18:13:22.471183: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-17 18:13:22.548080: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-17 18:13:22.885178: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-17 18:13:24.467477: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [21]:

# Building LSTM model
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(tf.keras.layers.LSTM(units=50))
model.add(tf.keras.layers.Dense(units=1))  # Output dimension is 1 for TAC prediction
model.compile(optimizer='adam', loss='mean_squared_error')


IndexError: tuple index out of range

In [ ]:

# Training the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)


In [ ]:

# Plotting training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:

# Making predictions
predicted_tac = model.predict(X_test)

# Rescale data back to original range
predicted_tac = scaler.inverse_transform(predicted_tac)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))


In [ ]:

# Plotting predictions
plt.figure(figsize=(10, 6))
plt.plot(y_test, label='Actual TAC')
plt.plot(predicted_tac, label='Predicted TAC')
plt.title('TAC Prediction')
plt.xlabel('Samples')
plt.ylabel('TAC')
plt.legend()
plt.show()
